In [4]:
# ─────────────────────────────────────────────────────────────────────────────
# Department & Office Comparison Bar Charts (saved alongside gauge folders)
# ─────────────────────────────────────────────────────────────────────────────
import pathlib, re, unicodedata
import numpy as np, pandas as pd, plotly.graph_objects as go

# 1) Paths (same as gauge script)
ROOT_DIR  = pathlib.Path(r"C:\Repositories\odi-data-visualization")
DATA_FILE = ROOT_DIR / "data" / "table_3.csv"
PLOTS_DIR = ROOT_DIR / "plots"   # <— same folder tree as gauges

# 2) Helpers (same behavior as your gauge script)
SAFE  = re.compile(r"[^0-9A-Za-z _()\-.’']+")
def normalize_quotes(s: str) -> str:
    s = unicodedata.normalize("NFKC", str(s))
    return s.replace("’", "'").strip()

def clean(s: str, n=80) -> str:
    s = normalize_quotes(s)
    s = SAFE.sub("_", s)
    s = re.sub(r"\s+", " ", s)
    return s[:n].strip()

BANDS = [
    (1.00, 3.29, "#e53935"),
    (3.30, 3.49, "#fb8c00"),
    (3.50, 3.79, "#fdd835"),
    (3.80, 5.00, "#43a047"),
]
def band_color(v, default="#999999"):
    try:
        v = float(v)
    except Exception:
        return default
    for lo, hi, col in BANDS:
        if lo <= v <= hi:
            return col
    return default

ELECTED_PATTERNS = (
    r"\bcity\s*auditor\b",
    r"\bthe\s*city\s*auditor\b",
    r"\bcity\s*attorney\b",
    r"\bcity\s*council\b",
)
def group_of(dept: str) -> str:
    norm = normalize_quotes(dept).casefold()
    if any(re.search(p, norm) for p in ELECTED_PATTERNS):
        return "elected_officials"
    return "mayors_office"

PRETTY_GROUP = {"elected_officials": "Elected Officials",
                "mayors_office": "Mayor’s Office"}

# 3) Data
df_raw = pd.read_csv(DATA_FILE, dtype=str)
df_raw.columns = [normalize_quotes(c) for c in df_raw.columns]
metric_col = df_raw.columns[0]
dept_cols  = df_raw.columns[1:]

df = df_raw.copy()
for c in dept_cols:
    df[c] = pd.to_numeric(df[c], errors="coerce")

score_rows = df.dropna(how="all", subset=dept_cols)
score_rows = score_rows[score_rows[metric_col] != "Outcome"]  # skip counts row
long = score_rows.melt(id_vars=[metric_col], value_vars=dept_cols,
                       var_name="department", value_name="score")
long["group"] = long["department"].apply(group_of)

office_avg = (long.groupby([metric_col, "group"], as_index=False)["score"]
              .mean().rename(columns={"score": "office_avg"}))
city_avg = (long.groupby([metric_col], as_index=False)["score"]
            .mean().rename(columns={"score": "city_avg"}))

# 4) Plot helpers
def bar_labels(values):
    return [f"{v:.2f}" if pd.notna(v) else "" for v in values]

def mk_bar_figure(x_labels, y_dept, y_comp, title, comp_name):
    colors = [band_color(v) for v in y_dept]
    dept_bar = go.Bar(
        x=x_labels, y=y_dept, name="Department",
        marker=dict(color=colors),
        text=bar_labels(y_dept), textposition="outside",
        offsetgroup=0, width=0.27,   # narrower for more gap between bars in same outcome
    )
    comp_bar = go.Bar(
        x=x_labels, y=y_comp, name=comp_name,
        marker=dict(
            color="#9E9E9E",
            pattern=dict(shape="/"),
            line=dict(color="#333333", width=2)  # semi-thick border
        ),
        text=bar_labels(y_comp), textposition="outside",
        offsetgroup=1, width=0.32,
    )
    fig = go.Figure([dept_bar, comp_bar])
    fig.update_layout(
        title=dict(text=title, x=0.5, y=0.94),
        barmode="group",
        bargap=0.3,                # bigger gap between outcomes
        plot_bgcolor="#FFFFFF",
        paper_bgcolor="#FFFFFF",
        legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="center", x=0.5),
        margin=dict(t=90, b=120, l=40, r=20),
        height=700, width=1100,
        yaxis=dict(
            range=[1, 5],
            tick0=1, dtick=1,
            title="Average score (1–5)"
        ),
        xaxis=dict(title=None, tickangle=-45),
    )
    return fig

def save_png(fig, path: pathlib.Path):
    path.parent.mkdir(parents=True, exist_ok=True)
    fig.write_image(str(path), scale=2)

# 5) Department charts → plots/<group>/<Department>/<file>.png
print("Saving department bar charts into the existing gauge folders…")
for dept in dept_cols:
    grp = group_of(dept)
    pretty_grp = PRETTY_GROUP[grp]
    sub = (long[long["department"] == dept]
           .merge(office_avg[office_avg["group"] == grp], on=metric_col, how="left")
           .dropna(subset=["score", "office_avg"])
           .sort_values(metric_col))

    x = sub[metric_col].tolist()
    y_dept = sub["score"].tolist()
    y_off  = sub["office_avg"].tolist()

    title = f"{dept} vs {pretty_grp} Average — Survey Subscales"
    fig = mk_bar_figure(x, y_dept, y_off, title, f"{pretty_grp} Average")

    # SAME folder structure as gauges:
    out_path = (PLOTS_DIR / grp / clean(dept) /
                f"{clean(dept)}_bars_vs_{clean(pretty_grp)}.png")
    save_png(fig, out_path)
    print("  •", out_path)

# 6) Office charts → plots/<group>/<Group>_bars_vs_Citywide.png
print("Saving office bar charts at the group root…")
for grp, pretty_grp in PRETTY_GROUP.items():
    sub = (office_avg[office_avg["group"] == grp]
           .merge(city_avg, on=metric_col, how="left")
           .dropna(subset=["office_avg", "city_avg"])
           .sort_values(metric_col))
    x = sub[metric_col].tolist()
    y_off  = sub["office_avg"].tolist()
    y_city = sub["city_avg"].tolist()

    title = f"{pretty_grp} vs Citywide Average — Survey Subscales"
    fig = mk_bar_figure(x, y_off, y_city, title, "Citywide Average")

    out_path = PLOTS_DIR / grp / f"{clean(pretty_grp)}_bars_vs_Citywide.png"
    save_png(fig, out_path)
    print("  •", out_path)

print("\nDone.")

Saving department bar charts into the existing gauge folders…
  • C:\Repositories\odi-data-visualization\plots\mayors_office\Building and Zoning Services\Building and Zoning Services_bars_vs_Mayor's Office.png
  • C:\Repositories\odi-data-visualization\plots\mayors_office\CelebrateOne\CelebrateOne_bars_vs_Mayor's Office.png
  • C:\Repositories\odi-data-visualization\plots\mayors_office\Civil Service\Civil Service_bars_vs_Mayor's Office.png
  • C:\Repositories\odi-data-visualization\plots\mayors_office\Columbus Public Health\Columbus Public Health_bars_vs_Mayor's Office.png
  • C:\Repositories\odi-data-visualization\plots\mayors_office\Development\Development_bars_vs_Mayor's Office.png
  • C:\Repositories\odi-data-visualization\plots\mayors_office\Diversity and Inclusion\Diversity and Inclusion_bars_vs_Mayor's Office.png
  • C:\Repositories\odi-data-visualization\plots\mayors_office\Finance and Management\Finance and Management_bars_vs_Mayor's Office.png
  • C:\Repositories\odi-data-vis